In [1]:
# Importing the required packages
using JuMP, HiGHS
using DataFrames, NamedArrays

# Name of different regions
location_name = [:calopia, :sorange, :tyran, :entworpe, :fardo]

# Expected total demand at each region
demand = Dict(zip(location_name, [28733, 51428, 11969, 11750, 10508]))

# Associated fixed costs

# Fixed cost to build a factory irrespective of capacity
fixed_factory = 500000

# Fixed cost to build a warehouse --> There is no upper bound to warehouse capacity, it has infinite capacity                     
fixed_warehouse = 100000

# variable costs
# Additional capcity costs of $50,000 per drum
variable_production = 50000   

# Holding cost per drum per year, includes both in transit and in warehouse
holding_warehouse = 100             

max_capacity = 73000


# Shipping cost from factory to warehouse
    # Cost per truck load: Same region ---> $15,000; 
    # Different region in same continent ---> $20,000 
    # Between Continent and Fardo ---> $45,000
    # Decided to ship full truck load of 200 drums per shipment, 
    # the cost matrix is calculated by dividing 200 drums respect to cost per truck load

shipping_factory_to_warehouse = [
    75 100 100 100 225
    100 75 100 200 225
    100 100 75 100 225
    100 100 100 75 225
    225 225 225 225 75
]

SFTW_NA = NamedArray(shipping_factory_to_warehouse, (location_name, location_name), 
                                                    ("Factory Location", "Warehouse Location"));

# Shipping cost from Warehouse to customer
# All orders are fulfilled through mail: Statement provided ---> Fixed can't change
        # Cost per drum: $150 for same region; 
        # $200 for different region in same continent; 
        # $400 for between Fardo

shipping_warehouse_to_customer = [
    150 200 200 200 400
    200 150 200 200 400
    200 200 150 200 400
    200 200 200 150 400
    400 400 400 450 150
]

SWTC_NA = NamedArray(shipping_warehouse_to_customer, (location_name, location_name), 
                                                     ("Warehouse Location", "Customer Location"));


# Declaring the model
model = Model(HiGHS.Optimizer)

# Variables

@variable(model, factory_decision[location_name], Bin)     # Binary variable if a factory is open or not
@variable(model, warehouse_decision[location_name], Bin)   # Binary variable if a warehouse is open or not


@variable(model, factory_capacity[location_name] >= 0)     # Factory capacity has to be greater than zero
@variable(model, warehouse_capacity[location_name] >= 0)   # Warehouse capacity has to be greater than zero


# Shipping quantity from factory to warehouse through truck
@variable(model, factory_to_warehouse_shipping[i in location_name, j in location_name] >= 0)


# Shipping quantity from warehouse to customer through mail
@variable(model, warehouse_to_customer[j in location_name, k in location_name] >= 0)


# Objective function as expression
@expression(model, factory_cost, sum(fixed_factory * factory_decision[i] for i in location_name))

@expression(model, warehouse_cost, sum(fixed_warehouse * warehouse_decision[j] for j in location_name))

@expression(model, production_cost, sum(variable_production * factory_capacity[i] for i in location_name))

@expression(model, shipping_cost_to_warehouse, 
    sum(SFTW_NA[i, j] * factory_to_warehouse_shipping[i, j] For i in location_name, j in location_name))

@expression(model, shipping_cost_to_customer, 
    sum(SWTC_NA[j, k] * warehouse_to_customer[j, k] for j in location_name, k in location_name))

@expression(model, holding_cost, 
    sum(holding_warehouse * warehouse_to_customer[j, k] for j in location_name, k in location_name))

# Final objective function
@objective(model, Min, factory_cost 
                    + warehouse_cost 
                    + production_cost 
                    + shipping_cost_to_warehouse 
                    + shipping_cost_to_customer 
                    + holding_cost)


# Constraints
# Initial Constraint
@constraint(model, factory_decision[:calopia] == 1)         # Calopia factory is already open at the start
@constraint(model, factory_capacity[:calopia] >= 20)        # Calopia has a initial production capacity


# Production  constraint
@constraint(model, [i in location_name], factory_capacity[i] <= (factory_decision[i] * max_capacity))


# Supply Constraint from factory to warehouse
@constraint(model, [i in location_name], 
    sum(factory_to_warehouse_shipping[i, j] for j in location_name) <= factory_capacity[i])


# Ship to warehouse from a factory if a warehouse is only open
# Including Big M Method, 
M = 200000000
@constraint(model, [j in location_name], 
    sum(factory_to_warehouse_shipping[i, j] for i in location_name) <= M * warehouse_decision[j])

# Flow balance constraint
@constraint(model, [j in location_name], 
    sum(factory_to_warehouse_shipping[i, j] for i in location_name) == sum(warehouse_to_customer[j, k] for k in location_name))

# Demand constraint
@constraint(model, [k in location_name], 
    sum(warehouse_to_customer[j, k] for j in location_name) == demand[k])


# Solving the problem
set_silent(model)
optimize!(model)


# Printing the results
optimal_cost = objective_value(model)
println("Optimal cost:", optimal_cost)

Optimal cost:5.759169075e9


In [2]:
# Print the factories that are open
factories_open = [i for i in location_name if value(factory_decision[i]) == 1]
println("Factories to Open:", factories_open)

Factories to Open:[:calopia, :sorange, :fardo]


In [3]:
# Print the warehouses that are open
warehouses_open = [i for i in location_name if value(warehouse_decision[i]) == 1]
println("Warehouses to Open:", warehouses_open)

Warehouses to Open:[:calopia, :sorange, :tyran, :entworpe, :fardo]


In [4]:
# Printing the factory capacities
factory_capacity_to_change = [(i, value(factory_capacity[i])*0.0016) for i in location_name if value(factory_capacity[i]) > 0]
println("Factory Capacities:", factory_capacity_to_change)

Factory Capacities:[(:calopia, 83.92320000000001), (:sorange, 82.2848), (:fardo, 16.8128)]


In [5]:
# Printing the shipment from factory to warehouse
println("Shipment from factory to warehouse:")
for i in location_name
    for j in location_name
        if value(factory_to_warehouse_shipping[i, j]) >= 1
            println("Factory $i serves Warehouse $j")
        end
    end
end

Shipment from factory to warehouse:
Factory calopia serves Warehouse calopia
Factory calopia serves Warehouse tyran
Factory calopia serves Warehouse entworpe
Factory sorange serves Warehouse sorange
Factory fardo serves Warehouse fardo


In [6]:
# Printing the shipment from warehouse to Customer
println("Shipment from warehouse to customer:")
for i in location_name
    for j in location_name
        if value(warehouse_to_customer[i, j]) >= 1
            println("Warehouse $i serves customers in $j")
        end
    end
end

Shipment from warehouse to customer:
Warehouse calopia serves customers in calopia
Warehouse sorange serves customers in sorange
Warehouse tyran serves customers in tyran
Warehouse entworpe serves customers in entworpe
Warehouse fardo serves customers in fardo
